In [42]:
import pandas.io.sql as psql
import pandas as pd
from datetime import datetime, timedelta
from sqlalchemy import create_engine
import requests
import json

# get data via API
data = json.loads(requests.get('https://min-api.cryptocompare.com/data/histoday?fsym=ETH&tsym=USD&limit=2000&aggregate=1').text)
df = pd.DataFrame(data["Data"])

# delete useless columns
df.drop(['close', 'open', 'volumefrom', 'volumeto'], axis=1, inplace = True)

#convert todatetime
df['time'] = pd.to_datetime(df['time'], unit='s')

# set index on column timestamp
df.set_index('time', inplace = True)

#drop rows with 0 only
columns = ['low', 'high']
df = df.replace(0, pd.np.nan).dropna(axis=0, how='any', subset=columns).fillna(0).astype(int)

In [48]:
# 15d/1m/3m/6m/12m/all
arr = [15, 30, 3*30, 6*30, 12*30, 2000]
arr_highs = [0, 0, 0, 0, 0, 0]
arr_lows = [0, 0, 0, 0, 0, 0]
date_after = datetime.now()
i = 0
for elt in arr:
    date_before = date_after - timedelta(days=elt)
    df_tmp = df.truncate(before=date_before, after=date_after)
    arr_highs[i] = df_tmp.high.max()
    arr_lows[i] = df_tmp.low.min()
    i+=1
print(arr_lows)
print(arr_highs)

[787, 564, 404, 200, 15, 0]
[981, 1155, 1422, 1422, 1422, 1422]
